## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


---
## Importing packages

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure, imshow, axis
import matplotlib.image as mpimg
import glob
import pickle
import warnings
from moviepy.editor import VideoFileClip
from HelperFunctions import *

%matplotlib inline

plt.rcParams.update({'figure.max_open_warning': 0})
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

---
## camera calibration using chessboard images

In [ ]:
def cameraImageUnDistort(mtx,dist,imageToUnDistort,showImages=False):
    
    dst = cv2.undistort(imageToUnDistort, mtx, dist, None, mtx)

    # Visualize undistortion
    if showImages:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.imshow(imageToUnDistort)
        ax1.set_title('Original Image')
        ax2.imshow(dst)
        ax2.set_title('Undistorted Image')
    return dst

In [ ]:
#camera calibration matrix and distortion coefficients 
def calibrationMatrixCoefficients(calibrationImagesPath,imageToCalibrate):
    #reading calibration images....
    # reading all the images
    images = glob.glob(calibrationImagesPath)

    #storing image point and object points
    objpoints = [] #3d points in real world space
    imgpoints = [] #2d points

    nx = 9
    ny = 6

    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
    
    for fname in images:
        img = mpimg.imread(fname)

        #converting image to Gray Scale
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)

        #finding the corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            check_img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
            
            # Visualize undistortion
            f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
            ax1.imshow(mpimg.imread(fname))
            ax1.set_title('Original Image')
            ax2.imshow(check_img)
            ax2.set_title('Dectected corners Image')
  
            
    # Calculate matrix and distortion coefficients 
    img = mpimg.imread(imageToCalibrate)
    img_size = (img.shape[1], img.shape[0])

    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    
    # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump( dist_pickle, open( "camera_cal/wide_dist_pickle.p", "wb" ) )
    
    return mtx, dist

In [ ]:
mtx, dist = calibrationMatrixCoefficients('camera_cal/calibration*.jpg','camera_cal/calibration4.jpg')

In [ ]:
def binary_Thresholding(img,showImages=False):

    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    sensitivity = 40
    color_ranges_white = [([0,0,255-sensitivity],[255,sensitivity,255]) ]
    color_ranges = [([5,50,50],[25,255,255]) ]
    color_ranges_yellow = [([5,100,100],[25,255,255])]
    
    output_yellow = None
    output_white = None
    
    #Getting Yellow lines from image
    for (lower, upper) in color_ranges_yellow :
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")
        mask = cv2.inRange(hsv,lower,upper)
        output_yellow = cv2.bitwise_and(img,img,mask = mask)
    
    #Getting White lines from image
    for (lower, upper) in color_ranges_white :
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")
        mask = cv2.inRange(hsv,lower,upper)
        output_white = cv2.bitwise_and(img,img,mask = mask)

    yellow_white = output_yellow + output_white
    
    yellow_white = cv2.cvtColor(yellow_white, cv2.COLOR_RGB2GRAY)
    

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = 50
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Threshold color channel
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1) | (yellow_white == 1)] = 1
    
    

    if showImages:
        # Plotting thresholded images
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('transformed image')
        ax1.imshow(img, cmap = 'gray')

        ax2.set_title('gradient thresholds')
        ax2.imshow(yellow_white + combined_binary, cmap='gray')
    
    return yellow_white + combined_binary

In [ ]:
# perspective transform images
def transform_image(img,showImages=False):
 
    img_size = (img.shape[1], img.shape[0])
 
    
    src = np.float32([[490, 482],[810, 482],
                      [1250, 720],[40, 720]])
    

    dst = np.float32([[0, 0], [1280, 0], 
                     [1250, 720],[40, 720]])
    
    
    
    M = cv2.getPerspectiveTransform(src, dst)
    
    Minv = cv2.getPerspectiveTransform(dst, src)
    
    warped = cv2.warpPerspective(img, M, img_size)
    
    if showImages:
        # Plotting thresholded images
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('Undistorted image')
        ax1.imshow(img, cmap='gray')

        ax2.set_title('Transformed image')
        ax2.imshow(warped, cmap='gray')
    
    return warped, M,Minv

In [ ]:
def calculateLaneLines(binary_warped,image,Minv,undistort,showimages=False):
    
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]/2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    
    
    
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    leftx_int = left_fit[0]*720**2 + left_fit[1]*720 + left_fit[2]
    
    rightx_int = right_fit[0]*720**2 + right_fit[1]*720 + right_fit[2]
    
    
    # Measure Radius of Curvature for each lane line
    ym_per_pix = 30./720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meteres per pixel in x dimension
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    left_curverad = ((1 + (2*left_fit_cr[0]*np.max(lefty) + left_fit_cr[1])**2)**1.5) \
                                 /np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*np.max(lefty) + right_fit_cr[1])**2)**1.5) \
                                    /np.absolute(2*right_fit_cr[0])
    
    
    # Calculate the position of the vehicle
    center = abs(640 - ((rightx_int+leftx_int)/2))
    
    
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (255,0, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undistort, 1, newwarp, 0.3, 0)
    
    
    if showimages:
        # Plotting thresholded images
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('Binary image')
        ax1.imshow(binary_warped, cmap='gray')

        ax2.set_title('lane detection image')
        ax2.imshow(out_img, cmap='gray')
        ax2.plot(left_fitx, ploty, color='yellow')
        ax2.plot(right_fitx, ploty, color='yellow')


        # Plotting thresholded images
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
        ax1.set_title('lane detection image')
        ax1.imshow(out_img, cmap='gray')

        ax2.set_title('Polonomial image')
        ax2.imshow(result)
        if center < 640:
            ax2.text(100, 100, 'Vehicle is {:.2f}m left of center'.format(center*3.7/700),
                     style='italic', color='white', fontsize=12)
        else:
            ax2.text(100, 100, 'Vehicle is {:.2f}m right of center'.format(center*3.7/700),
                     style='italic', color='white', fontsize=12)
        ax2.text(100, 155, 'Radius of curvature is {}m'.format(int((left_curverad + right_curverad)/2)),
                 style='italic', color='white', fontsize=12)
        
    else:
        font = cv2.FONT_HERSHEY_SIMPLEX
        if center < 640:
            cv2.putText(result, 'Vehicle is {:.2f}m left of center'.format(center*3.7/700), (100,100),
                     font,1, (255,255,255),2)
        else:
            cv2.putText(result, 'Vehicle is {:.2f}m right of center'.format(center*3.7/700), (100,100),
                     font,1, (255,255,255),2)
        # Print radius of curvature on video
        cv2.putText(result, 'Right Curvature {}(m)'.format(int((right_curverad))), (100,150),
                 font,1, (255,255,255),2)
        cv2.putText(result, 'Left Curvature {}(m)'.format(int((left_curverad))), (100,200),
                 font,1, (255,255,255),2)
        cv2.putText(result, 'Radius of Curvature {}(m)'.format(int((left_curverad + right_curverad)/2)), (100,250),
                 font,1, (255,255,255),2)
    
    return result
   
 
    
    

In [ ]:
def testPipline():
    images = glob.glob('test_images/test*.jpg')
    for fname in images:
        
        
        #reading image from the filename
        img = mpimg.imread(fname)
    
        
        dst = cameraImageUnDistort(mtx, dist, img,True)
        mpimg.imsave(fname.replace('.jpg','_dst.jpg').replace('test_images','output_images'),dst)
        
        warped,M,Minv = transform_image(dst,True)        
        mpimg.imsave(fname.replace('.jpg','_warped.jpg').replace('test_images','output_images'),warped)
        
        combined_binary = binary_Thresholding(warped,True)        
        mpimg.imsave(fname.replace('.jpg','_com.jpg').replace('test_images','output_images'),combined_binary)
        
        #print(fname.replace('.jpg','_com.jpg'))
        result = calculateLaneLines(combined_binary,img,Minv,dst,True)
       
        mpimg.imsave(fname.replace('.jpg','_result.jpg').replace('test_images','output_images'),result)
        #print(fname.replace('.jpg','_result.jpg'))
        
        


In [ ]:
testPipline()

In [ ]:
def videoPipline(img):
    
    
    dst = cameraImageUnDistort(mtx, dist, img)
        
    warped,M,Minv = transform_image(dst)
        
    combined_binary = binary_Thresholding(warped)

        
    result = calculateLaneLines(combined_binary,img,Minv,dst)
    
    
        
    return result

In [ ]:
video_output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image_Repeat) 
white_clip.write_videofile(video_output, audio=False)

In [ ]:
video_output = 'challenge_video_output.mp4'
clip1 = VideoFileClip("challenge_video.mp4")
white_clip = clip1.fl_image(process_image_Repeat) 
white_clip.write_videofile(video_output, audio=False)

In [ ]:
video_output = 'harder_challenge_video_output.mp4'
clip1 = VideoFileClip("harder_challenge_video.mp4")
white_clip = clip1.fl_image(process_image_Repeat) 
white_clip.write_videofile(video_output, audio=False)